# First Phase-Autoencoder

In [ ]:
seed = 1030#,1,2,3,4,55,666,7777,88888,999999
import random
random.seed(seed)
import numpy as np
np.random.seed(seed)

In [ ]:
#https://github.com/NeuroSyd/breast-cancer-sub-types
#https://gitlab.cs.washington.edu/abdincer/ad-ae
testPartitionIndex = 99

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir('/content/drive/My Drive/CNV')

In [ ]:
"""
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorflow_version 1
#Start tensorboard
%tensorboard — logdir logs
%load_ext autoreload
%autoreload 2
import warnings
warnings.simplefilter('ignore')
"""

In [ ]:
%matplotlib inline  

import imp
import numpy as np
import os
import keras
import keras.backend
import keras.models
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from keras import regularizers
from keras.models import Model,Sequential
from keras.layers import Input,Dense,Dropout
#from keras.layers.normalization import BatchNormalization
from tensorflow.keras.layers import BatchNormalization
from keras.optimizers import Adam, SGD, Adadelta
from keras.utils import plot_model
from keras import backend as K
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
import keras.models as mod
from keras.callbacks import EarlyStopping
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import TensorBoard
#from sklearn.externals import joblib
import joblib

In [ ]:

np.random.seed(seed)
df = pd.read_csv('./dataset/TCGA_BRCA_CNV.gz',compression='gzip',sep='\t',index_col=0)
df = df.reindex(sorted(df.columns), axis=1)
df=df.T
print("CNV Matrix Before:", df.shape)
df2=pd.read_csv('./dataset/BRCA_clinicalMatrix.gz',compression='gzip',sep='\t',index_col=0)
print("Clinical Data Before:", df2.shape)
k=df2.columns.get_loc('PAM50Call_RNAseq')
df2=df2[df2.iloc[:,k].isna()==False]             ########################3

################see pam50 included...!!!! try including all


###############################
#df2=df2[df2.iloc[:,k]!='Normal']
#d11=list(set(df.index)-set(df2.index)) #Comment this, if you want to test on unknown sample (generalize model)
commonIndexes=list(set(df2.index).intersection(set(df.index))) 
df=df.loc[commonIndexes]
df2=df2.loc[commonIndexes] 
#df=df.drop(d)                         #Comment this, if you want to test on unknown sample (generalize model)
print("CNV After:", df.shape)
print("Clinical Data After:", df2.shape)
#print("Gene Expression and Clinical Data corresponds to same Patients: ", False if False in (df.index==df2.index) else True)
######################unique_elements, counts_elements = np.unique(df2.iloc[:,k], return_counts=True)
#################print("Labels", unique_elements, counts_elements)

In [ ]:
df[df<-1]

In [ ]:
df.iloc[:,[2000]].hist()

In [ ]:
X=np.array(df.values)
#trainNoise = np.random.normal(loc=0.5, scale=0.5, size=trainX.shape)

scaler=StandardScaler()
XScaled=X#scaler.fit_transform(X)
print(XScaled.mean(axis = 0), XScaled.std(axis = 0))


################################33XScaled = X

In [ ]:
X.min(), X.max()

In [ ]:
plt.hist(XScaled[:,2000])

In [ ]:
#cols = [line.rstrip('\n') for line in open('./testPartitions/test'+str(testPartitionIndex)+'.txt')] 
#ptest= [df.index.get_loc(c) for c in cols]       #When checking whether the model is working or not, uncomemnt
ptest = []

In [ ]:
X.shape[0]

In [ ]:
X_test=XScaled[ptest,:]   #When checking whether the model is working or not, uncomemnt
n  = X.shape[0]
#X_test= np.asarray([])
ptrain=list(set(range(n))-set(ptest))
X_train=XScaled[ptrain,:]
X_test.shape,X_train.shape

In [ ]:
Y_temp=np.array(df2.values[:,k])
print("TARGET LABEL SIZE BEFORE: ", Y_temp.shape)
label_encoder=LabelEncoder()
Y=label_encoder.fit_transform(Y_temp)
print("TARGET LABEL SIZE BEFORE: ", Y.shape)
print("Labels before and after:", Y_temp[0:9], Y[0:9])
le_name_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(le_name_mapping)

In [ ]:
Y_train = Y.reshape(len(Y), 1)
Y_train.shape

In [ ]:
from imblearn.over_sampling import ADASYN
sm = ADASYN(random_state=seed)

#sm = SMOTE(random_state=seed)

print('Train Set before SMOTE:', X_train.shape, Y_train.shape)


X_train_sm, Y_train_sm = X_train, Y_train#sm.fit_resample(X_train, Y_train)
print('Train Set after SMOTE:', X_train_sm.shape, Y_train_sm.shape)

In [ ]:
unique_elements, counts_elements = np.unique(Y_train, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

In [ ]:
unique_elements, counts_elements = np.unique(Y_train_sm, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

In [ ]:

input_dim=X_train.shape[1]
encoding_dim=500


# sample noise from a random normal distribution centered at 0.5 (since
# our images lie in the range [0, 1]) and a standard deviation of 0.5

# corrupted_input = salt_and_pepper(input=x, corruption_level=0.4)


compression_factor=float(input_dim)/encoding_dim
print("Compression factor: %s" % compression_factor)

input_layer1=Input(shape=(input_dim,),name='input')
#hidden1=Dropout(0.0,name='dropout_100')(input_layer1)
hidden2=Dense(5000,activation='relu',name='dense_100')(input_layer1)

##################################################3hidden3=BatchNormalization(name='batch_normalization_100')(hidden2)#############33
#hidden31=Dropout(0.0,name='dropout_101')(hidden3)
hidden4=Dense(2000,activation='relu',name='dense_101')(hidden2)
##########################################hidden5=BatchNormalization(name='batch_normalization_101')(hidden4)##################
#hidden51=Dropout(0.0,name='dropout_102')(hidden5)
hidden10=Dense(encoding_dim,activation='relu',name='dense_105')(hidden4)
hidden7=BatchNormalization(name='batch_normalization_103')(hidden10) #############################3comment!!!!!
#hidden12=Dropout(0.2,name='dropout_103')(hidden11)
#hidden6=Dense(encoding_dim,activation='relu',name='dense_102')(hidden12)
#hidden7=BatchNormalization(name='batch_normalization_102')(hidden10)
#hidden13=Dense(500,activation='relu',name='dense_106')(hidden7)
hidden8=Dense(2000,activation='relu',name='dense_103')(hidden7)
hidden9=Dense(5000,activation='relu',name='dense_104')(hidden8)

output1=Dense(input_dim,activation='linear',name='output')(hidden9)     #####################linear
autoencoder=Model(inputs=input_layer1,outputs=output1)
autoencoder.summary()

In [ ]:
seed

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 classes=np.unique(np.argmax(Y_train, axis=-1)),
                                                y=np.argmax(Y_train, axis=-1))



In [ ]:
adam=Adam(lr=0.00002) #Change back to 0.00002
autoencoder.compile(optimizer=adam, loss='mean_squared_error',metrics=['accuracy'])   ###try binary_crossentropy
#es = EarlyStopping(monitor='val_loss', min_delta = 0.001, mode='min', verbose=1,patience=100)
#tbCallBack = TensorBoard(log_dir='./log', histogram_freq=1)
numIterations = 300       #For unknown test data, set it to 300.


In [ ]:
history=autoencoder.fit(X_train_sm, X_train_sm,
                epochs=numIterations,
                batch_size=80, #80 ##################################################################3
                shuffle=True#,  validation_split=0.0,   callbacks=[es] 
                )

plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
plt.title('Autoencoder Train loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper right')
plt.show()

#plt.savefig('./Results/autoencoderTrainloss.png')

In [ ]:
B=autoencoder.predict(X_train)
print("\nTraining Error %s: %.5f" % (autoencoder.metrics_names[1], (np.square(X_train-B)).mean(axis=None)))
#B=autoencoder.predict(X_test)
#print("\nTesting Error %s: %.5f" % (autoencoder.metrics_names[1], (np.square(X_test-B)).mean(axis=None)))
# 1000, no dropout, no noise
# Training Error accuracy: 0.00472
# Testing Error accuracy: 0.00885
# 2000, no dropout, no noise
# Training Error accuracy: 0.00465
# Testing Error accuracy: 0.00876 ##########################
# 2000, droupout 0.2, BatchNormalization
# Training Error accuracy: 0.00590
# Testing Error accuracy: 0.01520
# 2000, dropout 0, batchnormalization
# Training Error accuracy: 0.00381
# Testing Error accuracy: 0.01483
#2000, dropout 0.2, batchnorm after only middle layer


In [ ]:
B[0:10], X_train[0:10] #Training Error accuracy: 0.01813

In [ ]:
B[0:10], X_train[0:10] #Training Error accuracy: 0.01813

In [ ]:
encoder=Model(inputs=input_layer1,outputs=hidden10)
print(encoder.summary())

In [ ]:
#Encoder9: Correct Dat Without BN
#Encoder: BN after firsty two layers and not after 500!

 
model_json=encoder.to_json()
with open("./SavedModels/encoderBN"+str(seed)+".json","w") as json_file:
  json_file.write(model_json)
encoder.save_weights("./SavedModels/encoderBN"+str(seed)+".h5") 

In [ ]:
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx